In [1]:
import read_data as rd

In [2]:
import bleu_eval as be

In [3]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.optimizers import RMSprop, Adam
from keras.layers.wrappers import TimeDistributed

import random
import numpy as np

Using TensorFlow backend.


In [4]:
num_encoder_tokens = 4096
num_densed_tokens = 1024
latent_dim = 256
num_decoder_tokens = rd.max_dict_length
max_dict_length = rd.max_dict_length
max_sent_length = rd.max_sent_length

In [59]:
def input_generator(batch_size=64, option='all_sentence', encoder=None, decoder=None, scheduled_epoch=None):
    if option == 'all_sentence':
        total_movies = len(label.mvlist)
        curr_movie = 0
        curr_sent = 0
        while True:
            feat = np.array([]).reshape(0, 80, 4096)
            sent = np.array([]).reshape(0, max_sent_length, max_dict_length)
            for i in range(batch_size):
                if curr_sent >= len(label.mv2sent(label.mvlist[curr_movie % total_movies])):
                    curr_sent -= len(label.mv2sent(label.mvlist[curr_movie % total_movies]))
                # if curr_sent >= 1:
                #     curr_sent -= 1
                    curr_movie += 1
                feat = np.concatenate((feat, [label.mv2feat(label.mvlist[curr_movie % total_movies])]))
                sent = np.concatenate((sent, [label.sent2onehot(label.mv2sent(label.mvlist[curr_movie % total_movies])[curr_sent])]))
                curr_sent += 1
                
            yield [[feat, sent[:, :-1, :]], sent[:, 1:, :]]
    if option == 'random_each':
        total_movies = len(label.mvlist)
        curr_movie = 0
        while True:
            feat = np.array([]).reshape(0, 80, 4096)
            sent = np.array([]).reshape(0, max_sent_length, max_dict_length)
            for i in range(batch_size):
                # if curr_sent >= len(label.mv2sent(label.mvlist[curr_movie % total_movies])):
                #     curr_sent -= len(label.mv2sent(label.mvlist[curr_movie % total_movies]))
                # if curr_sent >= 1:
                #     curr_sent -= 1
                #     curr_movie += 1
                feat = np.concatenate((feat, [label.mv2feat(label.mvlist[curr_movie % total_movies])]))
                curr_sent = random.randint(0, len(label.mv2sent(label.mvlist[curr_movie % total_movies])) - 1)
                sent = np.concatenate((sent, [label.sent2onehot(label.mv2sent(label.mvlist[curr_movie % total_movies])[curr_sent])]))
                curr_movie += 1
                
            yield [[feat, sent[:, :-1, :]], sent[:, 1:, :]]
    if option == 'scheduled':
        if not encoder or not decoder:
            print('Error')
            return
        total_movies = len(label.mvlist)
        curr_movie = 0
        curr_epoch = 0
        while True:
            feat = np.array([]).reshape(0, 80, 4096)
            sent = np.array([]).reshape(0, max_sent_length, max_dict_length)
            for i in range(batch_size):
                # if curr_sent >= len(label.mv2sent(label.mvlist[curr_movie % total_movies])):
                #     curr_sent -= len(label.mv2sent(label.mvlist[curr_movie % total_movies]))
                # if curr_sent >= 1:
                #     curr_sent -= 1
                #     curr_movie += 1
                feat = np.concatenate((feat, [label.mv2feat(label.mvlist[curr_movie % total_movies])]))
                curr_sent = random.randint(0, len(label.mv2sent(label.mvlist[curr_movie % total_movies])) - 1)
                sent = np.concatenate((sent, [label.sent2onehot(label.mv2sent(label.mvlist[curr_movie % total_movies])[curr_sent])]))
                curr_movie += 1
            
            if curr_epoch >= scheduled_epoch:
                scheduled_sents = np.array([]).reshape(0, max_sent_length, max_dict_length)
                for input_seq, sample in zip(feat, sent):
                    scheduled_sent = np.zeros((1, max_sent_length, max_dict_length))
                    states_value = encoder.predict(input_seq.reshape(1, 80, 4096))
                    target_seq = np.zeros((1, 1, num_decoder_tokens))
                    target_seq[0, 0, label.word2int('<start>')] = 1.
                    
                    word_index = 0
                    scheduled_sent[0, word_index, :] = target_seq
                    word_index += 1
                    
                    stop_condition = False
                    while not stop_condition and word_index < max_sent_length-2:
                        output_tokens, h, c = decoder.predict(
                        [target_seq] + states_value)
                        sampled_token_index = np.random.choice(a=np.arange(num_decoder_tokens),
                                                               size=None,
                                                               replace=False,
                                                               p=output_tokens[0, -1, :])
#                         sampled_word = label.int2word(int(sampled_token_index))
#                         decoded_sentence += sampled_word
#                         if sampled_word == '<end>':
#                             stop_condition = True
                        if sampled_token_index == label.word2int('<end>'):
                            stop_condition = True
                        target_seq = np.zeros((1, 1, num_decoder_tokens))
                        target_seq[0, 0, sampled_token_index] = 1.
                        states_value = [h, c]
                        
                        scheduled_sent[0, word_index, :] = target_seq
                        word_index += 1
                        
                    for i in range(word_index, max_sent_length-1):
#                         scheduled_sent[0, i, :]
                        end_onehot = np.zeros((1, 1, num_decoder_tokens))
                        end_onehot[0, 0, label.word2int('<end>')] = 1.
                        scheduled_sent[0, i, :] = end_onehot
            
#                     print(scheduled_sents.shape, scheduled_sent.shape)
                    scheduled_sents = np.concatenate((scheduled_sents, scheduled_sent))
                yield [feat, scheduled_sents], sent[:, 1:, :]
                        
            else:
                yield [[feat, sent[:, :-1, :]], sent[:, 1:, :]]

In [60]:
gen = input_generator(option='scheduled', encoder=e, decoder=d, scheduled_epoch=200)
for i in range(5):
    [a, b], c = next(gen)
    print(a.shape, b.shape, c.shape)

(64, 80, 4096) (64, 49, 4000) (64, 49, 4000)
(64, 80, 4096) (64, 49, 4000) (64, 49, 4000)
(64, 80, 4096) (64, 49, 4000) (64, 49, 4000)
(64, 80, 4096) (64, 49, 4000) (64, 49, 4000)
(64, 80, 4096) (64, 49, 4000) (64, 49, 4000)


In [7]:
def set_model(mode='default'):

    if mode == 'default':
        # CNN_inputs = Input(shape=(None, num_encoder_tokens))
        # denser = TimeDistributed(Dense(num_densed_tokens, activation='relu'))
        # dense_outputs = denser(CNN_inputs)

        # encoder_inputs = Input(shape=(None, num_densed_tokens))
        encoder_inputs = Input(shape=(None, num_encoder_tokens))
        encoder = LSTM(latent_dim, return_state=True, implementation=2)
        encoder_outputs, state_h, state_c = encoder(encoder_inputs)
        # encoder_outputs, state_h, state_c = encoder(dense_outputs)
        encoder_states = [state_h, state_c]
        decoder_inputs = Input(shape=(None, num_decoder_tokens))
        decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, implementation=2)
        decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                             initial_state=encoder_states)
        decoder_dense = Dense(num_decoder_tokens, activation='softmax')
        decoder_outputs = decoder_dense(decoder_outputs)
        # model = Model([CNN_inputs, decoder_inputs], decoder_outputs)
        model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
        model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy')
        print(model.summary())

        encoder_model = Model(encoder_inputs, encoder_states)
        decoder_state_input_h = Input(shape=(latent_dim,))
        decoder_state_input_c = Input(shape=(latent_dim,))
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
        decoder_states = [state_h, state_c]
        decoder_outputs = decoder_dense(decoder_outputs)
        decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

        return model, encoder_model, decoder_model

In [8]:
m, e, d = set_model()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, 4096)    0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, None, 4000)    0                                            
____________________________________________________________________________________________________
lstm_1 (LSTM)                    [(None, 256), (None,  4457472     input_1[0][0]                    
____________________________________________________________________________________________________
lstm_2 (LSTM)                    [(None, None, 256), ( 4359168     input_2[0][0]                    
                                                                   lstm_1[0][1]            

In [10]:
DS = rd.Dataset('../hw2_dataset/training_data/feat/', '../hw2_dataset/training_label.json')
test_DS = rd.Dataset('../hw2_dataset/testing_data/feat/', '../hw2_dataset/testing_label.json')
label = DS.label

In [62]:
model_filename = 'models/1119_scheduled.h5'

In [22]:
def decode_sequence(input_seq, encoder_model, decoder_model):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, label.word2int('<start>')] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
#         print(output_tokens[0, -1, 73])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = label.int2word(int(sampled_token_index))
        decoded_sentence += sampled_word

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_word == '<end>':
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]
        decoded_sentence += ' ' 

    return decoded_sentence

In [17]:
def output_decoded_sequence(e, d, test=10):
    movies = label.mvlist
    for index, movie in enumerate(movies[:test]):
        input_seq = label.mv2feat(movie)
        decoded_sentence = decode_sequence(input_seq.reshape(1, 80, 4096), e, d)
        print('Decoded sentence: ', decoded_sentence)

In [64]:
def output_caption_txt(e, d):
    movies = test_DS.label.mvlist
    # with open('caption.txt', 'w') as f:

    result = {}
    for index, movie in enumerate(movies):
#         print(index, ':')
        input_seq = test_DS.label.mv2feat(movie)
        decoded_sentence = decode_sequence(input_seq.reshape(1, 80, 4096), e, d)
#         print('Video Name: ', movie)
#         print('Decoded Sentence: ', decoded_sentence)
        tmp = (decoded_sentence[:-7] + '.').capitalize()
        result[movie] = tmp
        # print(movie + ',' + tmp)
        # for sent in label.mv2sent(movie):
        #     print(sent)
        # f.write(movie + ',' + tmp + '\n')
    return result

In [19]:
def validation(result):
    return be.val_BLEU(result)

In [23]:
validation(output_caption_txt(e, d))

0 :
1 :


KeyboardInterrupt: 

In [ ]:
gen = input_generator(option='scheduled', encoder=e, decoder=d, scheduled_epoch=200)
max_score = 0
for i in range(150):
    m.fit_generator(gen, 4, epochs=50, verbose=1)
    # model.save_weights(model_filename)
    output_decoded_sequence(e, d)
    score = validation(output_caption_txt(e, d))
    if score > max_score:
        max_score = score
        m.save_weights(model_filename)
    print('This Epoch:', score, 'Current Max:', max_score)

Epoch 1/50
4/4 [==============================] - 5s - loss: 0.9027     
Epoch 2/50
4/4 [==============================] - 4s - loss: 0.8886     
Epoch 3/50
4/4 [==============================] - 4s - loss: 0.8653     
Epoch 4/50
4/4 [==============================] - 4s - loss: 0.9659     
Epoch 5/50
4/4 [==============================] - 4s - loss: 0.8762     
Epoch 6/50
4/4 [==============================] - 4s - loss: 0.8731     
Epoch 7/50
4/4 [==============================] - 4s - loss: 0.9076     
Epoch 8/50
4/4 [==============================] - 5s - loss: 0.9315     
Epoch 9/50
4/4 [==============================] - 5s - loss: 0.8832     
Epoch 10/50
4/4 [==============================] - 5s - loss: 0.9053     
Epoch 11/50
4/4 [==============================] - 5s - loss: 0.8661     
Epoch 12/50
4/4 [==============================] - 5s - loss: 0.9132     
Epoch 13/50
4/4 [==============================] - 5s - loss: 0.8901     
Epoch 14/50
4/4 [==============================

Decoded sentence:  a man is is a <end> 
Decoded sentence:  a man is is a a <end> 
Decoded sentence:  a man is is a <end> 
Originally, average bleu score is 0.22343473392488516
By another method, average bleu score is 0.4749444320230396
This Epoch: 0.4749444320230396 Current Max: 0.478130021556149
Epoch 1/50
4/4 [==============================] - 6s - loss: 0.7583     
Epoch 2/50
4/4 [==============================] - 6s - loss: 0.8566     
Epoch 3/50
4/4 [==============================] - 6s - loss: 0.7719     
Epoch 4/50
4/4 [==============================] - 6s - loss: 0.7970     
Epoch 5/50
4/4 [==============================] - 6s - loss: 0.7630     
Epoch 6/50
4/4 [==============================] - 6s - loss: 0.8070     
Epoch 7/50
4/4 [==============================] - 6s - loss: 0.7492     
Epoch 8/50
4/4 [==============================] - 6s - loss: 0.8073     
Epoch 9/50
4/4 [==============================] - 6s - loss: 0.8242     
Epoch 10/50
4/4 [============================

4/4 [==============================] - 5s - loss: 0.6999     
Epoch 50/50
4/4 [==============================] - 5s - loss: 0.7260     
Decoded sentence:  a man is playing a <end> 
Decoded sentence:  a man is cutting a <end> 
Decoded sentence:  a man is cutting a <end> 
Decoded sentence:  a man is playing a <end> 
Decoded sentence:  a man is playing a <end> 
Decoded sentence:  a man is playing a <end> 
Decoded sentence:  a man is playing a <end> 
Decoded sentence:  a man is cutting a <end> 
Decoded sentence:  a man is cutting a <end> 
Decoded sentence:  a man is playing a <end> 
Originally, average bleu score is 0.2227900331288974
By another method, average bleu score is 0.49352852736708863
This Epoch: 0.49352852736708863 Current Max: 0.5350013359782065
Epoch 1/50
4/4 [==============================] - 6s - loss: 0.7338     
Epoch 2/50
4/4 [==============================] - 5s - loss: 0.7093     
Epoch 3/50
4/4 [==============================] - 5s - loss: 0.7190     
Epoch 4/50
4/4 [=

4/4 [==============================] - 7s - loss: 0.6440     
Epoch 44/50
4/4 [==============================] - 7s - loss: 0.6934     
Epoch 45/50
4/4 [==============================] - 7s - loss: 0.6358     
Epoch 46/50
4/4 [==============================] - 7s - loss: 0.6904     
Epoch 47/50
4/4 [==============================] - 7s - loss: 0.6794     
Epoch 48/50
4/4 [==============================] - 7s - loss: 0.6454     
Epoch 49/50
4/4 [==============================] - 8s - loss: 0.6632     
Epoch 50/50
4/4 [==============================] - 7s - loss: 0.6683     
Decoded sentence:  a man is playing a <end> 
Decoded sentence:  a man is cutting a <end> 
Decoded sentence:  a woman is slicing a <end> 
Decoded sentence:  a man is playing a <end> 
Decoded sentence:  a man is playing a <end> 
Decoded sentence:  a man is playing a <end> 
Decoded sentence:  a man is playing a <end> 
Decoded sentence:  a woman is slicing a <end> 
Decoded sentence:  a man is cutting a <end> 
Decoded sen

4/4 [==============================] - 7s - loss: 0.6013     
Epoch 37/50
4/4 [==============================] - 7s - loss: 0.6216     
Epoch 38/50
4/4 [==============================] - 7s - loss: 0.6147     
Epoch 39/50
4/4 [==============================] - 7s - loss: 0.6266     
Epoch 40/50
4/4 [==============================] - 7s - loss: 0.6193     
Epoch 41/50
4/4 [==============================] - 7s - loss: 0.5710     
Epoch 42/50
4/4 [==============================] - 7s - loss: 0.6220     
Epoch 43/50
4/4 [==============================] - 7s - loss: 0.6212     
Epoch 44/50
4/4 [==============================] - 7s - loss: 0.5835     
Epoch 45/50
4/4 [==============================] - 7s - loss: 0.5991     
Epoch 46/50
4/4 [==============================] - 7s - loss: 0.6404     
Epoch 47/50
4/4 [==============================] - 7s - loss: 0.5942     
Epoch 48/50
4/4 [==============================] - 7s - loss: 0.6105     
Epoch 49/50
4/4 [==============================] -

...

4/4 [==============================] - 6s - loss: 0.3796     
Epoch 38/50
4/4 [==============================] - 6s - loss: 0.3861     
Epoch 39/50
4/4 [==============================] - 6s - loss: 0.3639     
Epoch 40/50
4/4 [==============================] - 6s - loss: 0.3624     
Epoch 41/50
4/4 [==============================] - 6s - loss: 0.3778     
Epoch 42/50
4/4 [==============================] - 6s - loss: 0.3886     
Epoch 43/50
4/4 [==============================] - 6s - loss: 0.3626     
Epoch 44/50
4/4 [==============================] - 6s - loss: 0.3931     
Epoch 45/50
4/4 [==============================] - 6s - loss: 0.3912     
Epoch 46/50
4/4 [==============================] - 6s - loss: 0.3795     
Epoch 47/50
4/4 [==============================] - 6s - loss: 0.3985     
Epoch 48/50
4/4 [==============================] - 6s - loss: 0.3849     
Epoch 49/50
4/4 [==============================] - 6s - loss: 0.3855     
Epoch 50/50
4/4 [==============================] -

4/4 [==============================] - 5s - loss: 0.3938     
Epoch 29/50
4/4 [==============================] - 5s - loss: 0.3840     
Epoch 30/50
4/4 [==============================] - 5s - loss: 0.3768     
Epoch 31/50
4/4 [==============================] - 5s - loss: 0.3713     
Epoch 32/50
4/4 [==============================] - 5s - loss: 0.3876     
Epoch 33/50
4/4 [==============================] - 5s - loss: 0.3694     
Epoch 34/50
4/4 [==============================] - 5s - loss: 0.3829     
Epoch 35/50
4/4 [==============================] - 5s - loss: 0.3226     
Epoch 36/50
4/4 [==============================] - 5s - loss: 0.3722     
Epoch 37/50
4/4 [==============================] - 5s - loss: 0.4044     
Epoch 38/50
4/4 [==============================] - 5s - loss: 0.3729     
Epoch 39/50
4/4 [==============================] - 5s - loss: 0.3574     
Epoch 40/50
4/4 [==============================] - 5s - loss: 0.3698     
Epoch 41/50
4/4 [==============================] -

4/4 [==============================] - 5s - loss: 0.3933     
Epoch 20/50
4/4 [==============================] - 5s - loss: 0.3743     
Epoch 21/50
4/4 [==============================] - 5s - loss: 0.3619     
Epoch 22/50
4/4 [==============================] - 5s - loss: 0.3631     
Epoch 23/50
4/4 [==============================] - 5s - loss: 0.4022     
Epoch 24/50
4/4 [==============================] - 5s - loss: 0.3898     
Epoch 25/50
4/4 [==============================] - 5s - loss: 0.3503     
Epoch 26/50
4/4 [==============================] - 5s - loss: 0.3777     
Epoch 27/50
4/4 [==============================] - 5s - loss: 0.3815     
Epoch 28/50
4/4 [==============================] - 5s - loss: 0.3331     
Epoch 29/50
4/4 [==============================] - 5s - loss: 0.3744     
Epoch 30/50
4/4 [==============================] - 5s - loss: 0.3734     
Epoch 31/50
4/4 [==============================] - 5s - loss: 0.3552     
Epoch 32/50
4/4 [==============================] -

4/4 [==============================] - 5s - loss: 0.4004     
Epoch 11/50
4/4 [==============================] - 5s - loss: 0.3815     
Epoch 12/50
4/4 [==============================] - 5s - loss: 0.3635     
Epoch 13/50
4/4 [==============================] - 5s - loss: 0.3622     
Epoch 14/50
4/4 [==============================] - 5s - loss: 0.3777     
Epoch 15/50
4/4 [==============================] - 5s - loss: 0.3418     
Epoch 16/50
4/4 [==============================] - 5s - loss: 0.3647     
Epoch 17/50
4/4 [==============================] - 5s - loss: 0.3582     
Epoch 18/50
4/4 [==============================] - 5s - loss: 0.3759     
Epoch 19/50
4/4 [==============================] - 5s - loss: 0.3643     
Epoch 20/50
4/4 [==============================] - 5s - loss: 0.3876     
Epoch 21/50
4/4 [==============================] - 5s - loss: 0.3547     
Epoch 22/50
4/4 [==============================] - 5s - loss: 0.3626     
Epoch 23/50
4/4 [==============================] -

4/4 [==============================] - 5s - loss: 0.3711     
Epoch 2/50
4/4 [==============================] - 5s - loss: 0.3992     
Epoch 3/50
4/4 [==============================] - 5s - loss: 0.3598     
Epoch 4/50
4/4 [==============================] - 5s - loss: 0.3910     
Epoch 5/50
4/4 [==============================] - 5s - loss: 0.3596     
Epoch 6/50
4/4 [==============================] - 5s - loss: 0.3846     
Epoch 7/50
4/4 [==============================] - 5s - loss: 0.3697     
Epoch 8/50
4/4 [==============================] - 5s - loss: 0.3707     
Epoch 9/50
4/4 [==============================] - 5s - loss: 0.3715     
Epoch 10/50
4/4 [==============================] - 5s - loss: 0.3772     
Epoch 11/50
4/4 [==============================] - 5s - loss: 0.3513     
Epoch 12/50
4/4 [==============================] - 5s - loss: 0.3884     
Epoch 13/50
4/4 [==============================] - 5s - loss: 0.3956     
Epoch 14/50
4/4 [==============================] - 5s - lo